In [44]:
%pip install gensim polars

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import polars as pl
import numpy as np
import gensim

In [3]:
# Load the osm.parquet file into a polars DataFrame
df = pl.read_parquet("../data/transformed/osm_clean.parquet")

In [4]:
df = df.with_columns(
    pl.col("name").fill_null(pl.col("alt_name")).fill_null(pl.col("description"))
)

In [48]:
# Filter the DataFrame to only include rows with POI data
poi_df = df.filter(
    pl.col("amenity").is_not_null()
    | pl.col("tourism").is_not_null()
    | pl.col("historic").is_not_null()
    | pl.col("leisure").is_not_null()
)

# Create a new column "poi" that combines the values of "amenity", "tourism", "leisure" and "historic"
poi_df = poi_df.with_columns(
    pl.when(pl.col("amenity").is_not_null())
    .then(pl.col("amenity"))
    .when(pl.col("tourism").is_not_null())
    .then(pl.col("tourism"))
    .when(pl.col("historic").is_not_null())
    .then(pl.col("historic"))
    .when(pl.col("leisure").is_not_null())
    .then(pl.col("leisure"))
    .alias("sub_category"),
    pl.when(pl.col("amenity").is_not_null())
    .then(pl.lit("amenity"))
    .when(pl.col("tourism").is_not_null())
    .then(pl.lit("tourism"))
    .when(pl.col("historic").is_not_null())
    .then(pl.lit("historic"))
    .when(pl.col("leisure").is_not_null())
    .then(pl.lit("leisure"))
    .alias("category"),
)

# drop the columns "amenity", "tourism", "historic" and "leisure"
poi_df = poi_df.drop(
    [
        "amenity",
        "tourism",
        "historic",
        "leisure",
    ]
)

In [49]:
poi_df = poi_df.filter(pl.col("rcn_ref").is_null())
poi_df = poi_df.drop(
    [
        "rcn_ref",
        "rce:ref",
        "contact:fax",
        "created_by",
        "ref",
        "button_operated",
        "crossing",
        "tactile_paving",
        "traffic_signals:sound",
        "traffic_signals:vibration",
        "fax",
        ".*:covid19$",
        "musical_instrument",
        "__index_level_0__",
        "source",
        "source:date",
        "network:type",
    ]
)

In [50]:
poi_df.write_parquet("../data/transformed/poi_clean_category.parquet")
poi_df.write_csv("../data/transformed/poi_clean_category.csv")